#### Summary:
This notebook will be used to process the raw ATAC data and call peaks (first unfiltered with Homer, then reproducible with IDR). I'll then use the H3K27ac data to call positive and negative peaks from both control and NASH treated data. Compared to previous iterations, I'll be raising our H3K27Ac threshold here to 32 (rather than 16).

Output files:
- strain_treat_annot.txt: original peak annotation file 
- strain_treat_filt_annot.txt: filtered peaks with H3K27ac > 16 and "intron" or "Intergenic"
- strain_treat_filt_annot_cut.bed: bed file of ^ with chrY and chrUn peaks removed

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import csv
import os

from scipy.stats.stats import pearsonr,spearmanr

from os import listdir
import fnmatch

New ATAC peaks from Hunter (circa end of Jan; I'm pretty sure I used these when making the round 2 peaks set? But maybe not bc then I'd expect it to NOT be completely identical to the round 1 set??):
- /gpfs/data01/glasslab/home/h1bennet/strains/data/ATAC/control/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119/
- /gpfs/data01/glasslab/home/h1bennet/strains/data/ATAC/control/00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916/
- /gpfs/data01/glasslab/home/h1bennet/strains/data/ATAC/AMLN_30week/00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425/
- /gpfs/data01/glasslab/home/h1bennet/strains/data/ATAC/AMLN_30week/00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919/

# Make Genome Browser Hub

Hunter changed these tag directories a bit so I remade these. BUT they still don't work, and I have no idea why not...

In [2]:
%%bash
cd /home/h1bennet/strains/data/ATAC/control/
tag_dir=$(ls -d *)
echo $tag_dir
makeMultiWigHub.pl C57Kupffer-ATAC-control3 mm10 -url http://homer.ucsd.edu/hubs/ -webdir /homer_data/www/html/hubs/ -d $tag_dir -force #&>/dev/null &

00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119 00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916 00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812 00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811 01_balbc_Kupffer_ATAC_control_young_balb10A_TDT_JSS_20161013 01_balbc_Kupffer_ATAC_control_young_balb10B_TDT_JSS_20161013 01_balbc_Kupffer_ATAC_control_young_balb10C_TDT_JSS_20161013 02_aj_Kupffer_ATAC_control_young_aj10A_TDT_JSS_20161013 02_aj_Kupffer_ATAC_control_young_aj10B_TDT_JSS_20161013 02_aj_Kupffer_ATAC_control_young_aj10C_TDT_JSS_20161013


Colors that will be used (change with -color or -gradient options):
	Index	Color	Neg. color	Tag Directory
	1	255,150,150	(255,180,180)	00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119
	2	150,150,255	(180,180,255)	00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916
	3	150,255,150	(180,255,180)	00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812
	4	255,200,150	(255,210,180)	00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811
	5	150,255,220	(180,255,240)	01_balbc_Kupffer_ATAC_control_young_balb10A_TDT_JSS_20161013
	6	200,150,255	(210,180,255)	01_balbc_Kupffer_ATAC_control_young_balb10B_TDT_JSS_20161013
	7	200,200,150	(210,210,170)	01_balbc_Kupffer_ATAC_control_young_balb10C_TDT_JSS_20161013
	8	150,200,200	(170,210,210)	02_aj_Kupffer_ATAC_control_young_aj10A_TDT_JSS_20161013
	9	200,150,200	(210,170,210)	02_aj_Kupffer_ATAC_control_young_aj10B_TDT_JSS_20161013
	10	50,251,25	(68,107,94)	02_aj_Kupffer_ATAC_control_young_aj10C_TDT_JSS_20161013


	Once finished, you will want t

In [3]:
%%bash
cd /home/h1bennet/strains/data/ATAC/AMLN_30week/
tag_dir=$(ls -d *)
echo $tag_dir
makeMultiWigHub.pl C57Kupffer-ATAC-AMLN3 mm10 -url http://homer.ucsd.edu/hubs/ -webdir /homer_data/www/html/hubs/ -d $tag_dir -force #&>/dev/null &

00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425 00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919 00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919 00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921 01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3C_JSS_TDT_160926 01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3D_JSS_TDT_160928 02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3A_JSS_TDT_160926 02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3B_JSS_TDT_160926


Colors that will be used (change with -color or -gradient options):
	Index	Color	Neg. color	Tag Directory
	1	255,150,150	(255,180,180)	00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425
	2	150,150,255	(180,180,255)	00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919
	3	150,255,150	(180,255,180)	00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919
	4	255,200,150	(255,210,180)	00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921
	5	150,255,220	(180,255,240)	01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3C_JSS_TDT_160926
	6	200,150,255	(210,180,255)	01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3D_JSS_TDT_160928
	7	200,200,150	(210,210,170)	02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3A_JSS_TDT_160926
	8	150,200,200	(170,210,210)	02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3B_JSS_TDT_160926


	Once finished, you will want to upload the following hub URL:
		http://homer.ucsd.edu/hubs//C57Kupffer-ATAC-AMLN3/hub.txt

	If loading to the Wash U Epigenome Browser, use:
		http://hom

# Call Peaks

## Step 1: Unfiltered peaks from HOMER

### Control

File names:
- 00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119/
- 00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916/

In [17]:
%%bash
cd /home/hmummey/data/ATAC/raw_peaks/Control
for data in ./*C57BL6*; do
    strain="$(cut -d '_' -f2 <<<"$data")"
    treatment="$(cut -d '_' -f5 <<<"$data")"    
    name="$(cut -d '/' -f2 <<<"$data")_round3"
    echo $name
    findPeaks $data -style factor -L 0 -C 0 -fdr 0.9 -o /home/hmummey/data/ATAC/init_peak_calling/${name}.txt -size 200 &>/dev/null &
done

00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3
00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3


In [6]:
tag_diretory_array = np.array(fnmatch.filter(listdir('/home/hmummey/data/ATAC/raw_peaks/Control'), '*C57BL6*'))
tag_diretory_array.sort()
tag_diretory_array

array(['00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119',
       '00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916'],
      dtype='<U57')

### AMLN

File names:
- 00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425/
- 00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919/

In [16]:
%%bash
cd /home/hmummey/data/ATAC/raw_peaks/AMLN_30week
for data in ./*AMLNDiet_2*; do
    strain="$(cut -d '_' -f2 <<<"$data")"
    treatment="$(cut -d '_' -f5 <<<"$data")"    
    name="$(cut -d '/' -f2 <<<"$data")_round3"
    echo $name
    findPeaks $data -style factor -L 0 -C 0 -fdr 0.9 -o /home/hmummey/data/ATAC/init_peak_calling/${name}.txt -size 200 &>/dev/null &
done

00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425_round3
00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919_round3


In [9]:
tag_diretory_array2 = np.array(fnmatch.filter(listdir('/home/hmummey/data/ATAC/raw_peaks/AMLN_30week'), '*AMLNDiet_2*'))
tag_diretory_array2.sort()
tag_diretory_array2

array(['00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425',
       '00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919'],
      dtype='<U60')

#### Creating a file with all chip_names (control and AMLN)

In [12]:
chip_names = []
for i in tag_diretory_array:
    strain = i.split('_')[1]
    treat = i.split('_')[4]
    chip_names.append('_'.join([strain, treat]))
    
for i in tag_diretory_array2:
    strain = i.split('_')[1]
    treat = i.split('_')[4]
    weeks = i.split('_')[5][0]
    chip_names.append('_'.join([strain, treat, weeks]))
    
chip_names = np.unique(np.array(chip_names))
chip_names

array(['C57BL6_control', 'NCoRWT_AMLNDiet_2'], dtype='<U17')

In [13]:
chip_file = '/home/hmummey/data/ATAC/chip_names_list_round3.txt' #exactly the same as prev file
with open(chip_file, 'w') as f:
    for c in chip_names:
        f.write(c + '\n')

## Step 2: Reproducible peaks from IDR

In [24]:
%%bash

cd /home/hmummey/data/ATAC/init_peak_calling
while IFS='' read -r IP
do
    strain="$(cut -d '_' -f1 <<<"$IP")"
    treatment="$(cut -d '_' -f2 <<<"$IP")"
    round="round3"
    #name="$(cut -d '/' -f2 <<<"$data" | cut -d '.' -f1)"

    echo $IP
    echo ./*${strain}*${treatment}*${round}*
    #echo *${strain}*${treatment}*
    echo 

    #mkdir ../IDR/${IP}_${round}
    
    python3 /home/zes017/run_idr_homerPeaks.py -threshold 0.05 ./*${strain}*${treatment}*${round}* ../IDR/${IP}_${round} #&>/dev/null &
done < '/home/hmummey/data/ATAC/chip_names_list_round3.txt'

C57BL6_control
./00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3.txt ./00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3.txt

Performing IDR analysis on the following samples: ./00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3.txt, ./00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3.txt
Output files will be written to: ../IDR/C57BL6_control_round3
Using the following IDR threshold: 0.05
Peaks will be ranked using: findPeaks Score
Other available scoreColumns: ['Normalized Tag Count' 'focus ratio' 'findPeaks Score' 'Score']
idr --samples ../IDR/C57BL6_control_round3/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3.narrowPeak ../IDR/C57BL6_control_round3/00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3.narrowPeak --output-file ../IDR/C57BL6_control_round3/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3_00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3_idr.out --plot 

Initial parameter values: [0.10 1.00 0.20 0.50]
Final parameter values: [2.26 1.42 0.92 0.65]
Number of reported peaks - 54774/94248 (58.1%)

Number of peaks passing IDR cutoff of 0.05 - 54774/94248 (58.1%)

/home/zes017/run_idr_homerPeaks.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrowPeakFrame['pValue'] = -1
Initial parameter values: [0.10 1.00 0.20 0.50]
Final parameter values: [1.90 1.34 0.87 0.62]
Number of reported peaks - 32825/65689 (50.0%)

Number of peaks passing IDR cutoff of 0.05 - 32825/65689 (50.0%)

/home/zes017/run_idr_homerPeaks.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

# RESUME HERE

# Count Tags of Histone Marks

In [28]:
%%bash

for peak_file in /home/hmummey/data/ATAC/annotated_peak_calling/round3/*_idr.tsv; do
    echo $peak_file

    #strain="$(echo $peak_file | cut -d '/' -f7 | cut -d '_' -f2)"
    strain="C57"
    treat="$(echo $peak_file | cut -d '/' -f8 | cut -d '_' -f5)"
    
    #treat (AMLNDiet) doesn't match his dir name: /home/h1bennet/strains/data/H3K27Ac/AMLN_30week
    #but it does match the file names: 00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423
    #so modifying this a bit (cut down and then use * later)
    treat2="$(echo $treat | cut -b 1-4)"

    echo $strain
    echo $treat
    echo $treat2
    
    files=''
    for h in 'H3K27Ac'; do
        files+="$(echo /home/h1bennet/strains/data/$h/$treat2*/*${strain}*${h}*)"

        files+=' '
    done
    echo $files
    echo
    annotatePeaks.pl $peak_file mm10 -size -500,500 -norm 1e7 -d $files > /home/hmummey/data/ATAC/annotated_peak_calling/round3/${strain}_${treat}_round3_annot.txt #2>/dev/null &
done

/home/hmummey/data/ATAC/annotated_peak_calling/round3/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3_00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3_idr.tsv
C57
control
cont
/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423

/home/hmummey/data/ATAC/annotated_peak_calling/round3/00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425_round3_00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919_round3_idr.tsv
C57
AMLNDiet
AMLN
/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423




	Peak file = /home/hmummey/data/ATAC/annotated_peak_calling/round3/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3_00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3_idr.tsv
	Genome = mm10
	Organism = mouse
	Peak Region set to 1000
	Will normalize tag counts to 1e7 per experiment
	Tag Directories:
		/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915
		/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 54773
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 54773
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Resizing peaks...
	Reading Positions...
	-----------------------

### Control

In [30]:
strain = 'C57_control_round3'
path = '/home/hmummey/data/ATAC/annotated_peak_calling/round3/'
annot_file = path + strain + '_annot.txt'
annot_df = pd.read_csv(annot_file, sep='\t')

In [31]:
print(len(annot_df))
annot_df.head()

54773


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/round3/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3_00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 Tag Count in 1000 bp (9468035.0 Total, normalization factor = 1.06, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423 Tag Count in 1000 bp (22615879.0 Total, normalization factor = 0.44, effective total = 1e7)"
0,457_20603,chr2,145952312,145953312,+,1000,63.0,"intron (NM_175280, intron 6 of 6)","intron (NM_175280, intron 6 of 6)",18028.0,...,78774.0,Mm.136347,NM_175280,ENSMUSG00000037143,Cfap61,4930529M08Rik|BB178539|BC024760,cilia and flagella associated protein 61,protein-coding,15.84,7.07
1,457_9613,chr4,123920875,123921875,+,1000,104.0,"intron (NM_017475, intron 3 of 6)","intron (NM_017475, intron 3 of 6)",3942.0,...,54170.0,Mm.220922,NM_017475,ENSMUSG00000028646,Rragc,AU041672|Gtr2|RAGC|TIB929|YGR163W,Ras-related GTP binding C,protein-coding,102.45,188.36
2,457_1301,chr5,110839354,110840354,+,1000,204.0,promoter-TSS (NM_016681),promoter-TSS (NM_016681),-77.0,...,100900.0,Mm.200231,NM_153571,ENSMUSG00000043510,Hscb,AI325508|AW049829|Hsc20,HscB iron-sulfur cluster co-chaperone,protein-coding,240.81,193.23
3,457_6548,chr14,105434993,105435993,+,1000,125.0,"intron (NR_045859, intron 2 of 2)",MTD|LTR|ERVL-MaLR,7934.0,...,71362.0,Mm.410781,NR_045859,ENSMUSG00000115548,5430440P10Rik,-,RIKEN cDNA 5430440P10 gene,ncRNA,35.91,54.39
4,457_4608,chr8,107029706,107030706,+,1000,144.0,Intergenic,Intergenic,-1120.0,...,116733.0,Mm.236004,NM_126165,ENSMUSG00000031913,Vps4a,4930589C15Rik|AI325971|AW553189,vacuolar protein sorting 4A,protein-coding,54.92,38.91


In [32]:
histone_tags1 = annot_df.iloc[:,-2:]
histone_tags1.columns = [i.split(' Tag')[0].split('/')[-1] for i in histone_tags1.columns.values]
histone_tags2 = np.log2(histone_tags1+1) #convert to log2 scale
histone_tags2.head()

,00_C57_Kupffer_H3K27Ac_control_young_C571A_170915,00_C57_Kupffer_H3K27Ac_control_young_C572A_180423
0,4.073820,3.012569
1,6.692790,7.564988
2,7.917730,7.601622
3,5.205940,5.791554
4,5.805292,5.318678


In [33]:
histone_tags2.describe()

,00_C57_Kupffer_H3K27Ac_control_young_C571A_170915,00_C57_Kupffer_H3K27Ac_control_young_C572A_180423
count,54773.000000,54773.000000
mean,4.989888,4.791526
std,1.985577,2.159701
min,0.000000,0.000000
25%,3.393691,3.090853
50%,5.121015,4.936873
75%,6.601548,6.559186
max,15.400366,14.988302


### AMLN Diet

In [34]:
strain2 = 'C57_AMLNDiet_round3'
path = '/home/hmummey/data/ATAC/annotated_peak_calling/round3/'
annot_file2 = path + strain2 + '_annot.txt'
annot_df2 = pd.read_csv(annot_file2, sep='\t')

print(len(annot_df2))
annot_df2.head()

32824


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/round3/00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425_round3_00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919_round3_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 Tag Count in 1000 bp (18749728.0 Total, normalization factor = 0.53, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423 Tag Count in 1000 bp (19231109.0 Total, normalization factor = 0.52, effective total = 1e7)"
0,868_10955,chr2,91537228,91538228,+,1000,62.0,"intron (NM_001165989, intron 1 of 43)","intron (NM_001165989, intron 1 of 43)",-8594.0,...,75786.0,Mm.168478,NM_029437,ENSMUSG00000040549,Ckap5,3110043H24Rik|4930432B04Rik|D730027C18Rik|T256...,cytoskeleton associated protein 5,protein-coding,18.13,25.48
1,868_8833,chr13,48967669,48968669,+,1000,69.0,promoter-TSS (NR_015601),promoter-TSS (NR_015601),57.0,...,68128.0,Mm.426304,NR_015601,NaN,Fam120aos,C030044B11Rik,"family with sequence similarity 120A, opposite...",ncRNA,155.74,108.16
2,868_14942,chr5,147860025,147861025,+,1000,52.0,promoter-TSS (NM_025624),promoter-TSS (NM_025624),-103.0,...,66537.0,Mm.332855,NM_025624,ENSMUSG00000029649,Pomp,2510048O06Rik|Ump1,proteasome maturation protein,protein-coding,192.54,162.24
3,868_16901,chr5,150522108,150523108,+,1000,48.0,promoter-TSS (NM_009765),promoter-TSS (NM_009765),-13.0,...,12190.0,Mm.236256,NM_009765,ENSMUSG00000041147,Brca2,Fancd1|RAB163,"breast cancer 2, early onset",protein-coding,83.20,49.92
4,868_13869,chr13,64401616,64402616,+,1000,54.0,Intergenic,Intergenic,-30437.0,...,105278.0,Mm.74982,NM_053180,ENSMUSG00000021483,Cdk20,4932702G04Rik|AU019450|AW558027|CDCH|Ccrk|PNQL...,cyclin-dependent kinase 20,protein-coding,42.13,40.04


In [35]:
histone_tags12 = annot_df2.iloc[:,-2:]
histone_tags12.columns = [i.split(' Tag')[0].split('/')[-1] for i in histone_tags12.columns.values]
histone_tags22 = np.log2(histone_tags12+1) #convert to log2 scale
histone_tags22.head()

,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423
0,4.257765,4.726831
1,7.292230,6.770300
2,7.596488,7.350851
3,6.395748,5.670161
4,5.430620,5.358959


In [36]:
histone_tags22.describe()

,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423
count,32824.000000,32824.000000
mean,5.459177,5.331023
std,1.922137,1.861231
min,0.000000,0.000000
25%,4.131754,4.007196
50%,5.691255,5.531069
75%,6.950352,6.756490
max,15.693564,15.991699


# Identify active enhancers

1) By setting an average threshold on H3K27ac tags > 32!!! (average 2 replicates, cutoff 32 on the actual tag counts, or 5 on log2 scale)

2) HOMER annotations to be at intronic or intergenic regions

### Control

In [37]:
histone_tags2.columns = ["C57_control_170915", "C57_control_180423"]
z = [((row.C57_control_170915 + row.C57_control_180423)/2) for index, row in histone_tags2.iterrows() ]
print(z[:5])
histone_tags2['C57_control_avg'] = z
histone_tags2.head()

[3.543194453397364, 7.128888817966269, 7.759676169332867, 5.498746732929741, 5.561985414668629]


,C57_control_170915,C57_control_180423,C57_control_avg
0,4.073820,3.012569,3.543194
1,6.692790,7.564988,7.128889
2,7.917730,7.601622,7.759676
3,5.205940,5.791554,5.498747
4,5.805292,5.318678,5.561985


In [38]:
#glancing at annotations
print((annot_df.iloc[:,7])[:10])

0     intron (NM_175280, intron 6 of 6)
1     intron (NM_017475, intron 3 of 6)
2              promoter-TSS (NM_016681)
3     intron (NR_045859, intron 2 of 2)
4                            Intergenic
5     intron (NM_008187, intron 1 of 5)
6              promoter-TSS (NM_007509)
7    intron (NM_008976, intron 3 of 18)
8                            Intergenic
9              promoter-TSS (NR_155747)
Name: Annotation, dtype: object


In [39]:
#just gonna loop through all rows (for both dfs) and record rows to save as those with:
#histone > 16 (average 2 replicates, cutoff 16 on the actual tag counts, or 4 on log2 scale)
#Annotation containing either: intron, Intergenic

save_rows = []
for row in np.arange(len(annot_df)):
    #grab full annotation
    full_annot = str(annot_df.iloc[row,7])
    if ("intron" in full_annot or "Intergenic" in full_annot) and (histone_tags2.iloc[row,2] > 5):
        save_rows.append(row)

In [40]:
print(len(save_rows))
print(save_rows[:10]) #looks good!

17539
[1, 3, 4, 5, 10, 11, 14, 18, 19, 20]


In [41]:
filt_annot = annot_df.iloc[save_rows,:]
print(len(filt_annot))
filt_annot.head()

17539


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/round3/00_C57BL6_Kupffer_ATAC_control_young_C5701_JSS_TDT_160119_round3_00_C57BL6_Kupffer_ATAC_control_young_C57E_JSS_TDT_170916_round3_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 Tag Count in 1000 bp (9468035.0 Total, normalization factor = 1.06, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423 Tag Count in 1000 bp (22615879.0 Total, normalization factor = 0.44, effective total = 1e7)"
1,457_9613,chr4,123920875,123921875,+,1000,104.0,"intron (NM_017475, intron 3 of 6)","intron (NM_017475, intron 3 of 6)",3942.0,...,54170.0,Mm.220922,NM_017475,ENSMUSG00000028646,Rragc,AU041672|Gtr2|RAGC|TIB929|YGR163W,Ras-related GTP binding C,protein-coding,102.45,188.36
3,457_6548,chr14,105434993,105435993,+,1000,125.0,"intron (NR_045859, intron 2 of 2)",MTD|LTR|ERVL-MaLR,7934.0,...,71362.0,Mm.410781,NR_045859,ENSMUSG00000115548,5430440P10Rik,-,RIKEN cDNA 5430440P10 gene,ncRNA,35.91,54.39
4,457_4608,chr8,107029706,107030706,+,1000,144.0,Intergenic,Intergenic,-1120.0,...,116733.0,Mm.236004,NM_126165,ENSMUSG00000031913,Vps4a,4930589C15Rik|AI325971|AW553189,vacuolar protein sorting 4A,protein-coding,54.92,38.91
5,457_19168,chr8,95427860,95428860,+,1000,67.0,"intron (NM_008187, intron 1 of 5)","intron (NM_008187, intron 1 of 5)",6509.0,...,14894.0,Mm.2080,NM_008187,ENSMUSG00000031796,Cfap20,2600014O15Rik|AL024127|Gtl3|T10-2A2,cilia and flagella associated protein 20,protein-coding,36.97,36.70
10,457_1618,chr8,27260710,27261710,+,1000,195.0,"intron (NM_007918, intron 1 of 2)","intron (NM_007918, intron 1 of 2)",883.0,...,13685.0,Mm.6700,NM_007918,ENSMUSG00000031490,Eif4ebp1,4e-bp1|AA959816|PHAS-I,eukaryotic translation initiation factor 4E bi...,protein-coding,102.45,97.72


In [42]:
#export this just in case
out_file = path + strain + "_filt_annot.txt"
filt_annot.to_csv(out_file, index=False)

### AMLN

In [43]:
histone_tags22.columns = ["C57_AMLN1", "C57_control_AMLN2"]
z2 = [((row.C57_AMLN1 + row.C57_control_AMLN2)/2) for index, row in histone_tags22.iterrows() ]
print(z2[:5])
histone_tags22['C57_AMLN_avg'] = z2
histone_tags22.head()

[4.49229809280194, 7.031265040702184, 7.473669381170035, 6.032954421152834, 5.394789318396052]


,C57_AMLN1,C57_control_AMLN2,C57_AMLN_avg
0,4.257765,4.726831,4.492298
1,7.292230,6.770300,7.031265
2,7.596488,7.350851,7.473669
3,6.395748,5.670161,6.032954
4,5.430620,5.358959,5.394789


In [44]:
#glancing at annotations
print((annot_df2.iloc[:,7])[:10])

0     intron (NM_001165989, intron 1 of 43)
1                  promoter-TSS (NR_015601)
2                  promoter-TSS (NM_025624)
3                  promoter-TSS (NM_009765)
4                                Intergenic
5    intron (NM_001033167, intron 10 of 10)
6                  promoter-TSS (NM_029711)
7      intron (NM_001310481, intron 4 of 5)
8         intron (NM_021297, intron 1 of 2)
9                                Intergenic
Name: Annotation, dtype: object


In [45]:
#just gonna loop through all rows (for both dfs) and record rows to save as those with:
#histone > 16 (average 2 replicates, cutoff 16 on the actual tag counts, or 4 on log2 scale)
#Annotation containing either: intron, Intergenic

save_rows2 = []
for row in np.arange(len(annot_df2)):
    #grab full annotation
    full_annot2 = str(annot_df2.iloc[row,7])
    if ("intron" in full_annot2 or "Intergenic" in full_annot2) and (histone_tags22.iloc[row,2] > 5):
        save_rows2.append(row)
        
print(len(save_rows2))
print(save_rows2[:10]) #looks good!

11600
[4, 5, 7, 8, 9, 13, 14, 18, 20, 21]


In [46]:
filt_annot2 = annot_df2.iloc[save_rows2,:]
print(len(filt_annot2))
filt_annot2.head()

11600


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/round3/00_NCoRWT_Kupffer_ATAC_AMLNDiet_20week_LN203D_JSS_TDT_180425_round3_00_NCoRWT_Kupffer_ATAC_AMLNDiet_21week_LN170C_JSS_TDT_160919_round3_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 Tag Count in 1000 bp (18749728.0 Total, normalization factor = 0.53, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423 Tag Count in 1000 bp (19231109.0 Total, normalization factor = 0.52, effective total = 1e7)"
4,868_13869,chr13,64401616,64402616,+,1000,54.0,Intergenic,Intergenic,-30437.0,...,105278.0,Mm.74982,NM_053180,ENSMUSG00000021483,Cdk20,4932702G04Rik|AU019450|AW558027|CDCH|Ccrk|PNQL...,cyclin-dependent kinase 20,protein-coding,42.13,40.04
5,868_20649,chr13,34187911,34188911,+,1000,41.0,"intron (NM_001033167, intron 10 of 10)","intron (NM_001033167, intron 10 of 10)",25447.0,...,69666.0,Mm.24788,NM_001101430,ENSMUSG00000071451,Psmg4,PAC-4|mPAC4,"proteasome (prosome, macropain) assembly chape...",protein-coding,79.47,80.08
7,868_18847,chr15,50761283,50762283,+,1000,44.0,"intron (NM_001310481, intron 4 of 5)","intron (NM_001310481, intron 4 of 5)",127266.0,...,83925.0,Mm.30466,NM_032000,ENSMUSG00000038679,Trps1,AI115454|AI447310|D15Ertd586e,transcriptional repressor GATA binding 1,protein-coding,33.60,40.56
8,868_15363,chr4,66828378,66829378,+,1000,51.0,"intron (NM_021297, intron 1 of 2)","intron (NM_021297, intron 1 of 2)",1327.0,...,21898.0,Mm.38049,NM_021297,ENSMUSG00000039005,Tlr4,Lps|Ly87|Ran/M1|Rasl2-8,toll-like receptor 4,protein-coding,270.94,241.80
9,868_3909,chr14,30592930,30593930,+,1000,97.0,Intergenic,MIR|SINE|MIR,16880.0,...,18753.0,Mm.2314,NM_011103,ENSMUSG00000021948,Prkcd,AI385711|D14Ertd420e|PKC[d]|PKCdelta|Pkcd,"protein kinase C, delta",protein-coding,355.74,367.11


In [47]:
#export this just in case
out_file2 = path + strain2 + "_filt_annot.txt"
filt_annot.to_csv(out_file2, index=False)

# Convert peak file to BED

Bed file format requires:
- chrom - name of the chromosome or scaffold. Any valid seq_region_name can be used, and chromosome names can be given with or without the 'chr' prefix.
- chromStart - Start position of the feature in standard chromosomal coordinates (i.e. first base is 0).
- chromEnd - End position of the feature in standard chromosomal coordinates

Optional columns: name, score, strand, thickStart, thickEnd, itemRgb, blockCount, blockSizes, blockStarts

from: https://m.ensembl.org/info/website/upload/bed.html

### Control

In [48]:
#just going to use the required cols (and maybe strand and score? why not, I guess)
save_cols = [1,2,3,4,5]
bed_df = filt_annot.iloc[:,save_cols]
print(len(bed_df))
bed_df.head()

17539


,Chr,Start,End,Strand,Peak Score
1,chr4,123920875,123921875,+,1000
3,chr14,105434993,105435993,+,1000
4,chr8,107029706,107030706,+,1000
5,chr8,95427860,95428860,+,1000
10,chr8,27260710,27261710,+,1000


In [49]:
chr_list = list(set(bed_df.iloc[:,0]))
chr_list.sort()
print(chr_list)

['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr4_GL456216_random', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrUn_JH584304', 'chrX', 'chrY']


Remove all rows that are chrY (makes the script to create the background, error) or chrUn

In [50]:
#also might as well try removing rows with Un chrs (will do this manually and see if helps)
keep_rows12 = []
for i in range(len(bed_df)):
    chr_str = bed_df.iloc[i,0]
    if ("chrY" in chr_str) or ("Un" in chr_str):
        pass
    else:
        keep_rows12.append(i)

print(len(keep_rows12))
print(keep_rows12[:20])

17500
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [51]:
bed_df_cut12 = bed_df.iloc[keep_rows12,:]
print(len(bed_df_cut12))
bed_df_cut12.head()

17500


,Chr,Start,End,Strand,Peak Score
1,chr4,123920875,123921875,+,1000
3,chr14,105434993,105435993,+,1000
4,chr8,107029706,107030706,+,1000
5,chr8,95427860,95428860,+,1000
10,chr8,27260710,27261710,+,1000


In [52]:
out3 = path + strain + "_filt_annot_fin.bed"
bed_df_cut12.to_csv(out3, index=False, sep='\t')

### AMLN

In [53]:
#just going to use the required cols (and maybe strand and score? why not, I guess)

save_cols2 = [1,2,3,4,5]
bed_df2 = filt_annot2.iloc[:,save_cols2]
print(len(bed_df2))
bed_df2.head()

11600


,Chr,Start,End,Strand,Peak Score
4,chr13,64401616,64402616,+,1000
5,chr13,34187911,34188911,+,1000
7,chr15,50761283,50762283,+,1000
8,chr4,66828378,66829378,+,1000
9,chr14,30592930,30593930,+,1000


In [54]:
chr_list = list(set(bed_df2.iloc[:,0]))
chr_list.sort()
print(chr_list)

['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr4_GL456216_random', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrUn_JH584304', 'chrX', 'chrY']


Remove all rows that are chrY (makes the script to create the background, error) or chrUn

In [55]:
#also might as well try removing rows with Un chrs (will do this manually and see if helps)
keep_rows22 = []
for i in range(len(bed_df2)):
    chr_str = bed_df2.iloc[i,0]
    if ("chrY" in chr_str) or ("Un" in chr_str):
        pass
    else:
        keep_rows22.append(i)
        
print(len(keep_rows22))
print(keep_rows22[:20])

11568
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [56]:
bed_df_cut22 = bed_df2.iloc[keep_rows22,:]
print(len(bed_df_cut22))
bed_df_cut22.head()

11568


,Chr,Start,End,Strand,Peak Score
4,chr13,64401616,64402616,+,1000
5,chr13,34187911,34188911,+,1000
7,chr15,50761283,50762283,+,1000
8,chr4,66828378,66829378,+,1000
9,chr14,30592930,30593930,+,1000


In [57]:
out4 = path + strain2 + "_filt_annot_fin.bed"
bed_df_cut22.to_csv(out4, index=False, sep='\t')

# Generate backgrounds sets

## Random matched-GC content sets

Bed file is the positives we've just chosen (only matching GC content). Make sure to remove the top row (col names) in terminal with sed '1d' before running the script! Col names are the same for all the C57 output files so not a huge loss to just delete them and then overwrite the file.

### Control

In [59]:
%%bash
bed_file="/home/hmummey/data/ATAC/annotated_peak_calling/round3/C57_control_round3_filt_annot_fin.bed" #+ peaks, no chrY, no chrUn
output=$(echo $bed_file | sed 's/.bed/_bg/g')
echo $output
python /home/zes017/Spacing/Codes/generate_background_coordinates.py $bed_file $output

/home/hmummey/data/ATAC/annotated_peak_calling/round3/C57_control_round3_filt_annot_fin_bg
reading genome mm10
fasta file does not exist: chr20
fasta file does not exist: chr21
fasta file does not exist: chr22
done reading genome
0 0
target GC: 0.37876971428569267 background GC: 0.38047438276007534 target length: 1000 numTargetPositions 1750 backgroundPositions 1750
0 0
target GC: 0.41272399999997644 background GC: 0.41207192807190457 target length: 1000 numTargetPositions 1750 backgroundPositions 1750
0 0
target GC: 0.43063999999997543 background GC: 0.42904524047378745 target length: 1000 numTargetPositions 1750 backgroundPositions 1750
0 0
target GC: 0.4453834285714032 background GC: 0.4416081061795096 target length: 1000 numTargetPositions 1750 backgroundPositions 1750
0 0
target GC: 0.45895542857140237 background GC: 0.4549216497787667 target length: 1000 numTargetPositions 1750 backgroundPositions 1750
0 0
target GC: 0.47263085714283015 background GC: 0.46738347366916133 target l

### AMLN --> skipping this step for now

In [60]:
#%%bash
#bed_file="/home/hmummey/data/ATAC/annotated_peak_calling/C57_AMLNDiet_round2_filt_annot_fin.bed" #+ peaks, no chrY, no chrUn
#output=$(echo $bed_file | sed 's/.bed/_bg/g')
#echo $output
#python /home/zes017/Spacing/Codes/generate_background_coordinates.py $bed_file $output

## Generate Differential Peak Sets

In [63]:
%%bash

#comparison of MY FINAL ANNOTATED PEAK SETS
#based off code in the notebook: 210124_Peak_comparison.ipynb

cd /home/hmummey/data/ATAC/annotated_peak_calling/round3/peak_comparison

file1="C57_AMLNDiet_round3_filt_annot_fin.bed"
file2="C57_control_round3_filt_annot_fin.bed"

echo $file1
echo $file2

mergePeaks -d given $file1 $file2 -prefix merged

C57_AMLNDiet_round3_filt_annot_fin.bed
C57_control_round3_filt_annot_fin.bed


	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing C57_AMLNDiet_round3_filt_annot_fin.bed (11568 total) and C57_AMLNDiet_round3_filt_annot_fin.bed (11568 total)
	Comparing C57_AMLNDiet_round3_filt_annot_fin.bed (11568 total) and C57_control_round3_filt_annot_fin.bed (17500 total)
	Comparing C57_control_round3_filt_annot_fin.bed (17500 total) and C57_AMLNDiet_round3_filt_annot_fin.bed (11568 total)
	Comparing C57_control_round3_filt_annot_fin.bed (17500 total) and C57_control_round3_filt_annot_fin.bed (17500 total)

C57_AMLNDiet_round3_filt_annot_fin.bed	C57_control_round3_filt_annot_fin.bed	Total	Name
	X	6221	C57_control_round3_filt_annot_fin.bed
X		1820	C57_AMLNDiet_round3_filt_annot_fin.bed
X	X	8343	C57_AMLNDiet_round3_filt_annot_fin.bed|C57_control_round3_filt_annot_fin.bed


## Generate Matched Size Bg (control) set for AMLN

In [2]:
#probably going to read in previous sets (with lower Ac to have larger training set)
#referring to peak comparison document to get differential peak files
#from the previous NCoRWT control vs. AMLN Diet

direct = '/home/hmummey/data/ATAC/annotated_peak_calling/peak_comparison/'
file1 = 'merge5_C57_control_round2_filt_annot_fin.bed'
file2 = 'merge5_C57_AMLNDiet_round2_filt_annot_fin.bed'
file3 = 'merge5_C57_AMLNDiet_round2_filt_annot_fin.bed_C57_control_round2_filt_annot_fin.bed'

In [5]:
print(direct+file1)
control_diff = pd.read_csv((direct+file1),sep='\t')
print(len(control_diff))
control_diff.tail()

/home/hmummey/data/ATAC/annotated_peak_calling/peak_comparison/merge5_C57_control_round2_filt_annot_fin.bed
6745


,#name (cmd = mergePeaks -d given C57_AMLNDiet_round2_filt_annot_fin.bed C57_control_round2_filt_annot_fin.bed -prefix merge5),chr,start,end,strand,Stat,Parent files,Total subpeaks,C57_AMLNDiet_round2_filt_annot_fin.bed,C57_control_round2_filt_annot_fin.bed
6740,Merged-chr11-5757997-1,chr11,5757498,5758497,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-4041
6741,Merged-chr3-136078820-1,chr3,136078321,136079320,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-4043
6742,Merged-chr1-21070186-1,chr1,21069687,21070686,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-4058
6743,Merged-chr13-111508828-1,chr13,111508329,111509328,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-5662
6744,Merged-chr13-104152560-1,chr13,104152061,104153060,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-5668


In [6]:
amln_diff = pd.read_csv((direct+file2),sep='\t')
print(len(amln_diff))
amln_diff.tail()

3966


,#name (cmd = mergePeaks -d given C57_AMLNDiet_round2_filt_annot_fin.bed C57_control_round2_filt_annot_fin.bed -prefix merge5),chr,start,end,strand,Stat,Parent files,Total subpeaks,C57_AMLNDiet_round2_filt_annot_fin.bed,C57_control_round2_filt_annot_fin.bed
3961,Merged-chr11-74522634-1,chr11,74522135,74523134,+,0.0,C57_AMLNDiet_round2_filt_annot_fin.bed,1,default-5654,NaN
3962,Merged-chr4-125127426-1,chr4,125126927,125127926,+,0.0,C57_AMLNDiet_round2_filt_annot_fin.bed,1,default-5656,NaN
3963,Merged-chr6-146271352-1,chr6,146270853,146271852,+,0.0,C57_AMLNDiet_round2_filt_annot_fin.bed,1,default-5658,NaN
3964,Merged-chr11-55131169-1,chr11,55130670,55131669,+,0.0,C57_AMLNDiet_round2_filt_annot_fin.bed,1,default-4056,NaN
3965,Merged-chr1-74308870-1,chr1,74308371,74309370,+,0.0,C57_AMLNDiet_round2_filt_annot_fin.bed,1,default-5660,NaN


In [8]:
#randomly sample 3,966 peaks from control first !!!!!!
import random

all_cnt_rows = list(range(len(control_diff)))
save_rows = random.sample(all_cnt_rows, len(amln_diff))
print(save_rows[:10])
save_rows.sort()
print(save_rows[:10])

[2912, 1870, 4352, 2798, 4552, 755, 2185, 3978, 2242, 2154]
[0, 3, 4, 5, 6, 8, 9, 10, 12, 15]


In [9]:
cut_control_diff = control_diff.iloc[save_rows,:]
print(len(cut_control_diff))
cut_control_diff.head()

3966


,#name (cmd = mergePeaks -d given C57_AMLNDiet_round2_filt_annot_fin.bed C57_control_round2_filt_annot_fin.bed -prefix merge5),chr,start,end,strand,Stat,Parent files,Total subpeaks,C57_AMLNDiet_round2_filt_annot_fin.bed,C57_control_round2_filt_annot_fin.bed
0,Merged-chr4-16169024-1,chr4,16168525,16169524,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-4062
3,Merged-chr13-55479188-1,chr13,55478689,55479688,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-4068
4,Merged-chr6-146172458-1,chr6,146171959,146172958,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-5671
5,Merged-chr17-53485827-1,chr17,53485328,53486327,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-5675
6,Merged-chr3-149085001-1,chr3,149084502,149085501,+,0.0,C57_control_round2_filt_annot_fin.bed,1,NaN,default-5678


In [11]:
#for training the models we need these as less complicated bed files in this format:
#chr, start, end, (anything else)
#so going to cut these down and then save in round3 folder

fin_file1 = '/home/hmummey/data/ATAC/annotated_peak_calling/round3/C57_control_diff_round2_filt_annot_fin.bed'
fin_file2 = '/home/hmummey/data/ATAC/annotated_peak_calling/round3/C57_AMLNDiet_diff_round2_filt_annot_fin.bed'

cut_control_diff.to_csv(fin_file1,index=False,header=False,columns=['chr','start','end'],sep='\t')
amln_diff.to_csv(fin_file2,index=False,header=False,columns=['chr','start','end'],sep='\t')